In [1]:
import pandas as pd # library for data manipulation and analysis.
import numpy as np # library for scientific computing, provides high-performance, easy to use structures and data analysis tools. 
import matplotlib.pyplot as plt # Plotting library
import seaborn as sns # data visualization library
sns.set(style="darkgrid")
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker import RandomCutForest
from sagemaker.model import Model
from sagemaker import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Step 2: Load data from S3 bucket
bucket = 'ccda-sgunda6'
prefix = "data"
file_name = 'personal-key-indicators-of-heart-disease.csv'
data_location = f's3://{bucket}/{prefix}/{file_name}'
data = pd.read_csv(data_location)
data

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,Unnamed: 0,State,Sex,GeneralHealth,LastCheckupTime,PhysicalActivities,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,...,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI
0,0,Alabama,Female,Very good,Within past year (anytime less than 12 months ...,No,None of them,No,No,No,...,No,"Yes, received tetanus shot but not sure what type",No,No,0.0,0.0,8.0,1.702691,83.074470,28.529842
1,1,Alabama,Female,Excellent,Within past year (anytime less than 12 months ...,No,None of them,No,No,No,...,No,"No, did not receive any tetanus shot in the pa...",No,No,0.0,0.0,6.0,1.600000,68.040000,26.570000
2,2,Alabama,Female,Very good,Within past year (anytime less than 12 months ...,Yes,None of them,No,No,No,...,No,"No, did not receive any tetanus shot in the pa...",No,Yes,2.0,3.0,5.0,1.570000,63.500000,25.610000
3,3,Alabama,Female,Excellent,Within past year (anytime less than 12 months ...,Yes,None of them,No,No,No,...,Yes,"No, did not receive any tetanus shot in the pa...",No,No,0.0,0.0,7.0,1.650000,63.500000,23.300000
4,4,Alabama,Female,Fair,Within past year (anytime less than 12 months ...,Yes,None of them,No,No,No,...,Yes,"No, did not receive any tetanus shot in the pa...",No,No,2.0,0.0,9.0,1.570000,53.980000,21.770000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890259,1890259,Arkansas,Female,Good,Within past year (anytime less than 12 months ...,Yes,None of them,No,No,No,...,No,"No, did not receive any tetanus shot in the pa...",No,Yes,0.0,0.0,3.0,1.550000,45.530896,19.012021
1890260,1890260,New York,Female,Good,Within past year (anytime less than 12 months ...,No,1 to 5,No,No,No,...,Yes,"Yes, received tetanus shot but not sure what type",No,No,0.0,0.0,7.0,1.520000,75.694060,32.151100
1890261,1890261,Massachusetts,Male,Good,Within past year (anytime less than 12 months ...,Yes,1 to 5,No,No,No,...,No,"No, did not receive any tetanus shot in the pa...",No,No,0.0,0.0,7.0,1.750000,108.237323,34.711666
1890262,1890262,Montana,Female,Very good,Within past 2 years (1 year but less than 2 ye...,Yes,None of them,No,No,No,...,No,"Yes, received Tdap",No,No,1.0,7.0,7.0,1.520000,56.977683,25.316571


In [3]:
data.columns

Index(['Unnamed: 0', 'State', 'Sex', 'GeneralHealth', 'LastCheckupTime',
       'PhysicalActivities', 'RemovedTeeth', 'HadHeartAttack', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
       'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos',
       'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours',
       'HeightInMeters', 'WeightInKilograms', 'BMI'],
      dtype='object')

In [4]:
# Step 3: Preprocess the data
categorical_features = ['State', 'Sex', 'GeneralHealth', 'LastCheckupTime', 'PhysicalActivities',
                         'RemovedTeeth', 'SmokerStatus', 'ECigaretteUsage', 'ChestScan', 
                         'RaceEthnicityCategory', 'AgeCategory', 'AlcoholDrinkers', 'HIVTesting', 
                         'FluVaxLast12', 'PneumoVaxEver', 'TetanusLast10Tdap', 'HighRiskLastYear', 
                         'CovidPos']

numerical_features = ['PhysicalHealthDays', 'MentalHealthDays', 'SleepHours', 'HeightInMeters',
                       'WeightInKilograms', 'BMI']

# Assuming 'HadHeartAttack' is the target variable
target_variable = 'HadHeartAttack'

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for feature in categorical_features:
    data[feature] = label_encoder.fit_transform(data[feature])

In [6]:
from sklearn.model_selection import train_test_split

X = data.drop(target_variable, axis=1)
y = data[target_variable]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [7]:
# Removing unwanted column
x_train = x_train.drop('Unnamed: 0', axis=1)
x_test = x_test.drop('Unnamed: 0', axis=1)

In [8]:
attributes = ['State', 'Sex', 'GeneralHealth', 'LastCheckupTime',
       'PhysicalActivities', 'RemovedTeeth', 'HadAngina',
       'HadStroke', 'HadAsthma', 'HadSkinCancer', 'HadCOPD',
       'HadDepressiveDisorder', 'HadKidneyDisease', 'HadArthritis',
       'HadDiabetes', 'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
       'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos',
       'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours',
       'HeightInMeters', 'WeightInKilograms', 'BMI']

trainX = pd.DataFrame(x_train, columns=attributes)
trainX['target'] = y_train

testX = pd.DataFrame(x_test, columns=attributes)
testX['target'] = y_test

In [9]:
trainX.head()

,State,Sex,GeneralHealth,LastCheckupTime,PhysicalActivities,RemovedTeeth,HadAngina,HadStroke,HadAsthma,HadSkinCancer,...,TetanusLast10Tdap,HighRiskLastYear,CovidPos,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI,target
1554490,19,1,1,3,0,3,No,No,No,No,...,2,1,0,30.0,30.0,6.0,1.78,112.797309,35.079780,No
208592,24,1,4,3,1,0,No,Yes,No,No,...,2,0,0,2.0,0.0,8.0,1.80,134.720000,41.420000,No
522370,21,1,1,3,0,0,No,No,No,No,...,2,0,2,30.0,0.0,15.0,1.91,123.619890,33.737369,No
33881,4,1,1,1,1,2,No,No,Yes,No,...,0,0,0,30.0,30.0,6.0,1.83,136.080000,40.690000,Yes
1378839,4,0,0,3,1,0,No,No,No,No,...,0,0,0,0.0,10.0,6.0,1.65,72.610302,26.673299,No


In [10]:
trainX.to_csv('heart_health_train.csv')
testX.to_csv('heart_health_test.csv')

In [11]:
import datetime
import tarfile

import boto3 # AWS SDK for python. Provides low-level access to AWS services
from sagemaker import get_execution_role
import sagemaker

m_boto3 = boto3.client('sagemaker') 

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = 'ccda-sgunda6'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [12]:
# send data to S3. SageMaker will take training data from s3
trainpath = sess.upload_data(
    path='heart_health_train.csv', bucket=bucket,
    key_prefix='data')

testpath = sess.upload_data(
    path='heart_health_test.csv', bucket=bucket,
    key_prefix='data')

In [20]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":
    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="heart_health_train.csv")
    parser.add_argument("--test-file", type=str, default="heart_health_test.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    


    print("building training and testing datasets")
    X_train = pd.get_dummies(train_df[args.features.split()], drop_first=True)
    X_test = pd.get_dummies(test_df[args.features.split()], drop_first=True)

    # Convert 'Yes'/'No' to 1/0 for binary classification
    y_train = train_df[args.target].map({'Yes': 1, 'No': 0})
    y_test = test_df[args.target].map({'Yes': 1, 'No': 0})



    #print("building training and testing datasets")
   # X_train = train_df[args.features.split()]
    #X_test = test_df[args.features.split()]
    #y_train = train_df[args.target]
    #y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)

Overwriting script.py


In [21]:

!python script.py --n-estimators 100 \
                  --min-samples-leaf 2 \
                  --model-dir ./ \
                  --train ./ \
                  --test ./ \
                  --features 'State Sex GeneralHealth LastCheckupTime PhysicalActivities RemovedTeeth HadAngina HadStroke HadAsthma HadSkinCancer HadCOPD HadDepressiveDisorder HadKidneyDisease HadArthritis HadDiabetes DeafOrHardOfHearing BlindOrVisionDifficulty DifficultyConcentrating DifficultyWalking DifficultyDressingBathing DifficultyErrands SmokerStatus ECigaretteUsage ChestScan RaceEthnicityCategory AgeCategory AlcoholDrinkers HIVTesting FluVaxLast12 PneumoVaxEver TetanusLast10Tdap HighRiskLastYear CovidPos PhysicalHealthDays MentalHealthDays SleepHours HeightInMeters WeightInKilograms BMI' \
                  --target target


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
extracting arguments
reading data
building training and testing datasets
training model
validating model
AE-at-10th-percentile: 0.0
AE-at-50th-percentile: 0.015
AE-at-90th-percentile: 0.21349999999999997
model persisted at ./model.joblib
2


In [22]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.c5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 3,
        "features": "State Sex GeneralHealth LastCheckupTime PhysicalActivities RemovedTeeth HadAngina HadStroke HadAsthma HadSkinCancer HadCOPD HadDepressiveDisorder HadKidneyDisease HadArthritis HadDiabetes DeafOrHardOfHearing BlindOrVisionDifficulty DifficultyConcentrating DifficultyWalking DifficultyDressingBathing DifficultyErrands SmokerStatus ECigaretteUsage ChestScan RaceEthnicityCategory AgeCategory AlcoholDrinkers HIVTesting FluVaxLast12 PneumoVaxEver TetanusLast10Tdap HighRiskLastYear CovidPos PhysicalHealthDays MentalHealthDays SleepHours HeightInMeters WeightInKilograms BMI",
        "target": "target",
    },
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [23]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: rf-scikit-2023-12-01-03-18-19-758


Using provided s3_resource
2023-12-01 03:18:20 Starting - Starting the training job...
2023-12-01 03:18:37 Starting - Preparing the instances for training.........
2023-12-01 03:19:59 Downloading - Downloading input data...
2023-12-01 03:20:49 Training - Training image download completed. Training in progress..2023-12-01 03:20:51,328 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-12-01 03:20:51,331 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-01 03:20:51,370 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-12-01 03:20:51,540 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-01 03:20:51,550 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-01 03:20:51,561 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-12-01 03:20:51,569 sagemaker-training-toolki

In [46]:
# we use the Hyperparameter Tuner
from sagemaker.tuner import IntegerParameter

# Define exploration boundaries
hyperparameter_ranges = {
    "n-estimators": IntegerParameter(20, 100),
    "min-samples-leaf": IntegerParameter(2, 6),
}

# create Optimizer
Optimizer = sagemaker.tuner.HyperparameterTuner(
    estimator=sklearn_estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name="RF-tuner",
    objective_type="Minimize",
    objective_metric_name="median-AE",
    metric_definitions=[
        {"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}
    ],  # extract tracked metric from logs with regexp
    max_jobs=10,
    max_parallel_jobs=2,
)

In [47]:
Optimizer.fit({"train": trainpath, "test": testpath})

Using provided s3_resource


INFO:sagemaker:Creating hyperparameter tuning job with name: RF-tuner-231201-0415


...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [48]:
# get tuner results in a df
results = Optimizer.analytics().dataframe()
while results.empty:
    time.sleep(1)
    results = Optimizer.analytics().dataframe()
results.head()

,min-samples-leaf,n-estimators,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,3.0,46.0,RF-tuner-231201-0415-010-cfcc34e0,Completed,0.014493,2023-12-01 04:53:46+00:00,2023-12-01 05:00:19+00:00,393.0
1,3.0,68.0,RF-tuner-231201-0415-009-c327091d,Completed,0.014706,2023-12-01 04:46:22+00:00,2023-12-01 04:55:16+00:00,534.0
2,2.0,47.0,RF-tuner-231201-0415-008-7408b2e2,Completed,0.014184,2023-12-01 04:46:22+00:00,2023-12-01 04:52:55+00:00,393.0
3,3.0,49.0,RF-tuner-231201-0415-007-bec91c8b,Completed,0.014286,2023-12-01 04:38:55+00:00,2023-12-01 04:45:38+00:00,403.0
4,3.0,47.0,RF-tuner-231201-0415-006-d707fa70,Completed,0.014184,2023-12-01 04:38:48+00:00,2023-12-01 04:45:21+00:00,393.0


In [49]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = m_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-12-01 03:34:14 Starting - Preparing the instances for training
2023-12-01 03:34:14 Downloading - Downloading input data
2023-12-01 03:34:14 Training - Training image download completed. Training in progress.
2023-12-01 03:34:14 Uploading - Uploading generated training model
2023-12-01 03:34:14 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-518154354065/rf-scikit-2023-12-01-03-18-19-758/output/model.tar.gz


In [50]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [51]:
predictor = model.deploy(instance_type="ml.c5.large", initial_instance_count=1)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-12-01-05-01-08-958
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2023-12-01-05-01-09-594
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2023-12-01-05-01-09-594


----!

In [52]:
testX.columns

Index(['State', 'Sex', 'GeneralHealth', 'LastCheckupTime',
       'PhysicalActivities', 'RemovedTeeth', 'HadAngina', 'HadStroke',
       'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder',
       'HadKidneyDisease', 'HadArthritis', 'HadDiabetes',
       'DeafOrHardOfHearing', 'BlindOrVisionDifficulty',
       'DifficultyConcentrating', 'DifficultyWalking',
       'DifficultyDressingBathing', 'DifficultyErrands', 'SmokerStatus',
       'ECigaretteUsage', 'ChestScan', 'RaceEthnicityCategory', 'AgeCategory',
       'AlcoholDrinkers', 'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
       'TetanusLast10Tdap', 'HighRiskLastYear', 'CovidPos',
       'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours',
       'HeightInMeters', 'WeightInKilograms', 'BMI', 'target'],
      dtype='object')

In [53]:
# Make the prediction
result = predictor.predict(testX)

# Process the prediction result
print("Prediction Result:", result)

ClientError: An error occurred (413) when calling the InvokeEndpoint operation: 